In [2]:
import gdown
import pandas as pd
import sqlite3

In [70]:
# Se cargan archivos CSV en DataFrames

users_df = pd.read_csv("users.csv")
loans_df = pd.read_csv("loans.csv")
payments_df = pd.read_csv("payments.csv")

In [4]:
# Crear una conexión a la base de datos SQLite

conn = sqlite3.connect("my_database.db")

In [71]:
# Se guardan los DataFrames en la base de datos

users_df.to_sql("users", conn, index=False, if_exists="replace")
loans_df.to_sql("loans", conn, index=False, if_exists="replace")
payments_df.to_sql("payments", conn, index=False, if_exists="replace")

1000

In [72]:
# La variable query siempre contendra la consulta 
query = """
SELECT *
FROM users
LIMIT 5;
"""

# Ejecutar la consulta y mostrar los resultados
result = pd.read_sql(query, conn)
result

,user_id,created_dt,name,email,phone,birthdate,gender,street_address,cp,country
0,1,2022-10-09,Jason Williams,coxjessica@example.com,5827692878,1965-06-22,Female,285 Heather Center Suite 394,44214,Antigua and Barbuda
1,2,2023-01-09,Craig Rivera,lauraruiz@example.com,1734940118,1972-07-25,Female,4446 Young Fork Suite 838,44150,American Samoa
2,3,2022-05-30,Mrs. Natasha White MD,lkoch@example.org,6961590041,1953-04-20,Male,3172 Meyer Summit Apt. 765,44150,Ecuador
3,4,2023-02-18,Sara Hill,brittanyhenry@example.com,2348607979,2001-12-07,Male,66651 Elizabeth Trail,44214,Suriname
4,5,2021-06-07,Scott Young,reyesryan@example.net,4276638595,1967-06-29,Female,91775 Melissa Flat Apt. 330,44219,Japan


In [73]:
# La variable query siempre contendra la consulta 
query = """
SELECT *
FROM loans
LIMIT 5;
"""

# Ejecutar la consulta y mostrar los resultados
result = pd.read_sql(query, conn)
result

,loan_id,user_id,contract_start_date,contract_end_date,loan_amount,interest_rate,payment_frequency,loan_term
0,1,6,2023-04-24,2024-01-26,27757.0,0.02,monthly,26
1,2,2,2023-05-14,2023-08-14,10996.0,0.08,monthly,12
2,3,7,2023-08-08,2023-09-19,26199.0,0.02,monthly,51
3,4,9,2023-07-24,2024-03-19,44504.0,0.04,monthly,35
4,5,3,2023-08-26,2024-05-18,21924.0,0.11,monthly,28


In [74]:
# La variable query siempre contendra la consulta 
query = """
SELECT *
FROM payments
LIMIT 5;
"""

# Ejecutar la consulta y mostrar los resultados
result = pd.read_sql(query, conn)
result

,payment_id,user_id,loan_id,limit_payment_dt,payment_date,payment_method,hash_card,issue_bank,payment_amount
0,1,3,6,2023-05-20,2023-08-19,Cash,2298106694523871,BBVA,475.0
1,2,4,2,2023-03-11,2023-01-12,Debit Card,3573234539022869,BBVA,189.0
2,3,8,10,2023-08-01,2023-03-30,Cash,4978763984748651267,Banamex,735.0
3,4,5,1,2023-06-05,2023-03-11,Cash,4316475747284821,Santander,4093.0
4,5,7,5,2023-01-07,2023-07-09,Cash,379874840799853,BBVA,265.0


In [75]:
# a. Select all customers that have multiple active loans

query = """
SELECT a.user_id, b.name, count(loan_id) AS active_loans

FROM loans a
JOIN users b 
ON a.user_id = b.user_id

WHERE contract_end_date > DATE('now')

GROUP BY 1,2
HAVING active_loans > 1;
"""
result = pd.read_sql(query, conn)
result

,user_id,name,active_loans
0,1,Jason Williams,83
1,2,Craig Rivera,80
2,3,Mrs. Natasha White MD,58
3,4,Sara Hill,72
4,5,Scott Young,78
5,6,Andrew Cobb,58
6,7,Aaron Martinez,63
7,8,Mark Jensen,64
8,9,Brian Stone,85
9,10,Ricardo Davis,66


In [76]:
# b. Select all customers that have made two consecutive payments of any of their active loans.
query = """
WITH date_active_payments AS (

SELECT a.user_id, payment_date, a.payment_id, LAG(payment_date) OVER (partition by a.user_id ORDER BY payment_date ASC) AS lag_date
       
 
FROM payments a
JOIN loans b
ON a.loan_id = b.loan_id

WHERE contract_end_date > DATE('now')

),

date_diff AS(

SELECT user_id, COUNT(payment_id) AS payments_made, AVG(julianday(payment_date)- julianday(lag_date)) AS diff_payments_avg

FROM date_active_payments

GROUP BY 1
)

SELECT a.user_id, b.payments_made, b.diff_payments_avg, COUNT(a.loan_id) AS total_loan,  
       MAX(julianday(DATE('now'))- julianday(contract_start_date)) AS user_age

FROM loans a
LEFT JOIN date_diff b
ON a.user_id = b.user_id

WHERE a.contract_end_date > DATE('now')

GROUP BY 1,2,3

"""
result = pd.read_sql(query, conn)
result

,user_id,payments_made,diff_payments_avg,total_loan,user_age
0,1,56,4.090909,83,238.0
1,2,82,2.851852,80,237.0
2,3,61,3.900000,58,236.0
3,4,75,3.189189,72,237.0
4,5,64,3.698413,78,239.0
5,6,79,2.948718,58,238.0
6,7,79,3.025641,63,239.0
7,8,85,2.773810,64,237.0
8,9,52,4.372549,85,237.0
9,10,74,3.178082,66,239.0


In [77]:
# c. Select the total amount paid and the average number of payments made by customers that have only one active loan.

query = """

WITH active_loan AS (
    SELECT user_id, COUNT(loan_id) AS active_loans
    FROM loans 
    WHERE contract_end_date > DATE('now')
    GROUP BY user_id
    HAVING active_loans = 1
)

SELECT a.user_id, b.name, SUM(c.payment_amount) AS total_paid_loan,
       100*(COUNT(c.payment_id)/(SELECT COUNT(*) FROM payments WHERE user_id = a.user_id)) AS average_payment_only_one

FROM active_loan a
JOIN users b 
ON a.user_id = b.user_id
JOIN payments c 
ON a.user_id = c.user_id

GROUP BY a.user_id, b.name;
"""
result = pd.read_sql(query, conn)
result


,user_id,name,total_paid_loan,average_payment_only_one


In [81]:
# d. Select the total amount paid grouped by zip code and age buckets
query = """

WITH users_generation AS (
SELECT user_id, ROUND((julianday(DATE('now'))- julianday(birthdate))/365,0) AS years_old, cp, 
       strftime('%Y',birthdate) AS year_born,
       CASE WHEN CAST(strftime('%Y',birthdate) as integer) BETWEEN 1945 AND 1964 THEN 'baby_boomers' 
       WHEN CAST(strftime('%Y',birthdate) as integer) BETWEEN 1965 AND 1981 THEN 'x_generation'
       WHEN CAST(strftime('%Y',birthdate) as integer) BETWEEN 1982 AND 1996 THEN 'millenial'
       WHEN CAST(strftime('%Y',birthdate) as integer) BETWEEN 1997 AND 2010 THEN 'z_generation' ELSE 'other' END AS generations

FROM users )

SELECT a.cp, generations, SUM(payment_amount) AS tota_payment_amount, COUNT(DISTINCT a.user_id) AS users

FROM  users_generation a
LEFT JOIN payments b
ON a.user_id = b.user_id

GROUP BY 1,2

"""
result = pd.read_sql(query, conn)
result

,cp,tota_payment_amount,users
0,44150,839965.0,188
1,44214,718815.0,207
2,44219,497790.0,196
3,44248,NaN,179
4,44266,509352.0,230
